In [148]:
from math import *
import numpy as np
import scipy.special

In [149]:
#math operations

def IsInteger(x):
    return x == floor(x)

def binomial(n,k):
    return scipy.special.comb(n,k, exact = True)

#list operations

def AreIntegers(a):
    for x in a:
        if not IsInteger(x):
            return False
    return True

def IsNegative(a):
    for x in a:
        if x < 0:
            return True
    return False

def HasPositiveIndex(a):
    for x in a:
        if x >0:
            return True
    return False  

def sum_list(list1,list2):
    return [list1[i] + list2[i] for i in range(len(list1))]

def minus_list(list1,list2):
    return [list1[i] - list2[i] for i in range(len(list1))]

def scalar_list(a,list1):
    return [a*x for x in list1]

def dot(list1,list2):
    return sum([list1[i] * list2[i] for i in range(len(list1))])

def E(i,n):
    l = [0 for i in range(n)]
    l[i-1] = 1
    return l
def nE(i,n):
    l = [0 for i in range(n)]
    l[i-1] = -1
    return l
    
def Zeros(n):
    return [0 for i in range(n)]

def ConstantList(d,n):
    return [d for i in range(n)]

def ListProd(L1,L2):
   return [[a,b] for a in L1 for b in L2]    

#Some coefficients operations

def ImaginarySign(d,a,b):
    if not mu(d,a,b)%2:
        return 1
    else:
        return 1j

def mu(d,a,b):
    return 3*d-sum(a)-2*sum(b)

def kU(d,a,b,l):
    return mu(d,a,b)-2*l-1

def lU(d,a,b,k):
    return (mu(d,a,b)-k-1)//2

def lF(d,a):
    return 3*d - sum(a) - 1 

def lFinv(d,a,b):
    return mu(d,a,b)-1

def OOSign(d1,a1,b1,d2,a2,b2,d,a,b):
    if d1+d2 != d or sum_list(a1,a2) != a or sum_list(b1,b2) != b:
        print("OOSign Error, parameters: ", d1, a1, b1, d2, a2, b2, d, a, b)
        return
    return ImaginarySign(d1, a1, b1)*ImaginarySign(d2, a2, b2)/ImaginarySign(d, a, b)

def OCSign(dU,aU,bU,d,a,b):
    return ImaginarySign(dU, aU, bU)/ImaginarySign(d, a, b)

#Some precompute list

def SmallerMultiIndices(a):
    tail = a[1::]
    if len(tail) == 0:
        return [[i] for i in range(a[0]+1)]
    return [[i,*b] for i in range(a[0]+1) for b in SmallerMultiIndices(tail)]   

def exeptionalDivisors(r):
    return [nE(i+1,r) for i in range(r)]

def nexeptionalDivisors(r):
    return [E(i+1,r) for i in range(r)]


def effective(d,a):
    r = len(a)
    if not IsInteger(d) or not AreIntegers(a) or d<0 or IsNegative(a):
        return False
    
    if (d == 0  and a in exeptionalDivisors(r)) or (d == 1 and a == Zeros(r)):
        return True

    if d == 1:
        for i in range(r-1):
            for j in range(i+1,r):
                if a == sum_list(E(i+1,r),E(j+1,r)):
                    return True

    for i in range(r-1):
        if a[i] > d:
            return False;
        for j in range(i+1,r):
            if a[i]+a[j] > d:
                return False
    return True

def effectiveInv(d,a,b):
    r = len(a)
    s = len(b)

    if not IsInteger(d) or not AreIntegers(a) or not AreIntegers(b) or d<0 or IsNegative(a) or IsNegative(b):
        return False
    if d == 0 and a in exeptionalDivisors(r) and b == Zeros(s):
        return True

    if d == 1:
        if ((a == Zeros(r) and b == Zeros(s)) or (a in list(-np.array(exeptionalDivisors(s))) and b == Zeros(s))
        or (a == Zeros(r) and b in list(-2*np.array(exeptionalDivisors(s))))):
            return True
        for i in range(1,r):
            for j in range(i+1,r+1):
                if a == sum_list(E(i,r),E(j,r)):
                    return True

    for i in range(r):
        for j in range(r):
            if i != j and a[i] + a[j] > d:
                return False
    return True

def MultiIndicesBetween(a,b):
    aTail = a[1::]
    bTail = b[1::]
    if len(a) != len(b):
        return []
    if len(aTail) == 0:
        return [[i] for i in range(int(a[0]),int(b[0])+1)]
    return [[i,*tailList] for i in range(int(a[0]),int(b[0])+1) for tailList in MultiIndicesBetween(aTail, bTail)]

def mem(f):
    memory = {}
    def inner_function(d,a):
        if not (d,tuple(a)) in memory:
            memory[(d,tuple(a))] = f(d,a)
            return memory[(d,tuple(a))]
        else:
            return memory[(d,tuple(a))]
    return inner_function

def mem_dub(f):
    memory = {}
    def inner_function(d,a,b,k):
        if not (d,tuple(a),tuple(b),k) in memory:
            memory[(d,tuple(a),tuple(b),k)] = f(d,a,b,k)
            return memory[(d,tuple(a),tuple(b),k)]
        else:
            return memory[(d,tuple(a),tuple(b),k)]
    return inner_function

@mem
def GW1(d,a):
    l = lF(d,a)
    S = 0
    for d1 in range(1,d):
        d2 = d-d1
        for a1 in SmallerMultiIndices(a):
            a2 = minus_list(a,a1)
            l1 = lF(d1,a1)
            l2 = lF(d2,a2)
            coef = (d1*d2-dot(a1, a2))*(d1*d2*binomial(l-3, l1-1)-d1**2*binomial(l-3, l1)) 
            S += coef*GW(d1, a1)*GW(d2, a2)       
    return S
@mem
def GW2(d,a):
    S = 0
    l = lF(d,a)
    r = len(a)
    i = 0
    while a[i] == 0:
        i+= 1
    coef = d**2-(a[i]-1)**2
    S += coef*GW(d, minus_list(a,E(i+1, r)))
    for d1 in range(1,d):
        d2 = d-d1
        for a1 in SmallerMultiIndices(a):
            a2 = minus_list(minus_list(a,a1),E(i+1, r))
            l1 = lF(d1,a1)
            l2 = lF(d2,a2)
            coef = (d1*d2-dot(a1, a2))*(d1*d2*a1[i]*a2[i]-d1**2*a2[i]**2)*binomial(l, l1)
            S += coef*GW(d1, a1)*GW(d2, a2)
    return S/(d**2*a[i])
@mem
def GW(d,a):
    r = len(a)
    if not effective(d,a):
        return 0
    if d == 0:
        if a in exeptionalDivisors(r):
            return 1
        return 0

    if d == 1 and a == Zeros(r):
        return 1
    l = lF(d,a)

    if l >= 3:
        return GW1(d,a)
    if l >= 0:
        return GW2(d,a)
    return 0

def invGW(d,a,b):
    S = 0
    cList = (0.5*np.array(MultiIndicesBetween(list(2*np.array(ConstantList(-d,len(b)))+2*np.array(b)),
                                                  list(2*np.array(ConstantList(d,len(b)))-2*np.array(b))))).tolist()

    for c in cList:
        S += GW(d,a+sum_list(b,c)+minus_list(b,c))
    return S






def OGW2(d,a,b,k):
    r = len(a)
    s = len(b)
    l = lU(d,a,b,k)
    if  l < 1 or k < 1:
        print("OGW2(", d, a, b, k, ") with l=", l)
        return 0
    total = 0
    for d1 in range(1,d):
        d2 = d - d1
        for a1 in SmallerMultiIndices(a):
            a2 = minus_list(a,a1)
            for b1 in SmallerMultiIndices(b):
                b2 = minus_list(b,b1)
                for k1 in range(k+1):
                    k2 = k-k1+1
                    l1 = lU(d1,a1,b1,k1)
                    l2 = lU(d2,a2,b2,k2)
                    if IsInteger(l1) and IsInteger(l2):
                        coef = (1/2)*OOSign(d1, a1, b1, d2, a2, b2, d, a, b)*binomial(l-1, l1)*d1*((1/2)*binomial(k-1, k1-1)*d2-(1/2)*binomial(k-1, k1)*d1)
                        if coef != 0:
                            total += coef*OGW(d1, a1, b1, k1)*OGW(d2, a2, b2, k2)

    for dU in range(d+1):
        dF = (d-dU)*(1/2)
        for aU in exeptionalDivisors(r) or aU in SmallerMultiIndices(a):
            aF = list(1/2*np.array(minus_list(a,aU)))
            for bU in exeptionalDivisors(s) or bU in SmallerMultiIndices(b):
                bF = list(1/2*np.array(minus_list(b,bU)))
                lu = lU(dU,aU,bU,k)
                lfInv = lFinv(dF,aF,bF)
                if IsInteger(dF) and AreIntegers(aF) and IsInteger(lu) and IsInteger(lfInv):
                    if dU >= 0 and dF >= 0 and IsNegative(aF) and IsNegative(bF) and lu >= 0 and lfInv >= 0:
                        coef = OCSign(dU, aU, bU, d, a, b)*binomial(l-1, lfInv)*dF**2*((1/2)*dF*dU-(1/2)*dot(aF, aU)-dot(bF, bU))
                        if coef != 0:
                            total -= coef*invGW(dF, aF, bF)*OGW(dU, aU, bU, k)
    if k == 1:
        total -= (1/4)*d**2*invGW((1/2)*d, list((1/2)*np.array(a)), list((1/2)*np.array(b)))
    return total

def OGW3a(d,a,b,k):
    r = len(a)
    s = len(b)
    l = lU(d,a,b,k)
    i = 0
    while a[i] == 0:
        i+= 1
    
    if l < 1 or i >= r:
        print("OGW4(", d, a, k, ") with l=", l)
        return 0
    total = 0

    for d1 in range(1,d):
        d2 = d-d1
        for a1 in SmallerMultiIndices(a):
            a2 = minus_list(a,a1)
            for b1 in SmallerMultiIndices(b):
                b2 = minus_list(b,b1)
                for k1 in range(k+1):
                    k2 = k-k1+1
                    l1 = lU(d1,a1,b1,k1)
                    l2 = lU(d2,a2,b2,k2)
                    if IsInteger(l1) and IsInteger(l2):
                        coef = (1/2)*OOSign(d1, a1, b1, d2, a2, b2, d, a, b)*binomial(k, k1)*binomial(l-1, l1)*d1*(-(1/4)*d2*a1[i]+(1/4)*d1*a2[i])
                        if coef != 0:
                            total += coef*OGW(d1, a1, b1, k1)*OGW(d2, a2, b2, k2)
    for dU in range(d+1):
        dF = list(1/2 * np.array(minus_list(d,dU)))
        for aU in exeptionalDivisors(r) or aU in SmallerMultiIndices(a):
            aF = list(1/2 * np.array(minus_list(a,aU)))
            for bU in exeptionalDivisors(s) or bU in SmallerMultiIndices(a):
                bF = list(1/2 * np.array(minus_list(b,bU)))
                lu = lU(dU,aU,bU,k)
                lfInv = lFinv(dF,aF,bF)
                if IsInteger(dF) and AreIntegers(aF) and IsInteger(lu) and IsInteger(lfInv):
                    if dU >= 0 and dF >= 0 and IsNegative(aF) and IsNegative(bF) and lu >= 0 and lfInv >= 0:
                        coef = OCSign(dU, aU, bU, d, a, b)*binomial(l-1, lfInv)*dF*((1/2)*aF[i]*dU-(1/2)*dF*aU[i])*(-(1/2)*dF*dU+(1/2)*dot(aF, aU)+dot(bF, bU))
                        if coef != 0:
                            total += coef*invGW(dF, aF, bF)*OGW(dU, aU, bU, k)
    return (-2/a[i])*total


def OGW3b(d,a,b,k):
    r = len(a)
    s = len(b)
    l = lU(d,a,b,k)
    j = 0
    while b[j] == 0:
        j += 1
    if l < 1 or j >= s:
        print("OGW3b(", d, a, k, ") with l=", l)
        return 0
    total = 0

    for d1 in range(1,d):
        d2 = d-d1
        for a1 in SmallerMultiIndices(a):
            a2 = minus_list(a,a1)
            for b1 in SmallerMultiIndices(b):
                b2 = minus_list(b,b1)
                for k1 in range(k+1):
                    k2 = k-k1+1
                    l1 = lU(d1,a1,b1,k1)
                    l2 = lU(d2,a2,b2,k2)
                    if IsInteger(l1) and IsInteger(l2):
                        coef = (1/2)*OOSign(d1, a1, b1, d2, a2, b2, d, a, b)*binomial(k, k1)*binomial(l-1, l1)*d1*(-(1/4)*d2*b1[j]+(1/4)*d1*b2[j])
                        if coef != 0:
                            total += coef*OGW(d1, a1, b1, k1)*OGW(d2, a2, b2, k2)
    for dU in range(d+1):
        dF = list(1/2 * np.array(minus_list(d,dU)))
        for aU in exeptionalDivisors(r) or aU in SmallerMultiIndices(a):
            aF = list(1/2 * np.array(minus_list(a,aU)))
            for bU in exeptionalDivisors(s) or bU in SmallerMultiIndices(a):
                bF = list(1/2 * np.array(minus_list(b,bU)))
                lu = lU(dU,aU,bU,k)
                lfInv = lFinv(dF,aF,bF)
                if IsInteger(dF) and AreIntegers(aF) and IsInteger(lu) and IsInteger(lfInv):
                    if dU >= 0 and dF >= 0 and IsNegative(aF) and IsNegative(bF) and lu >= 0 and lfInv >= 0:
                        coef = OCSign(dU, aU, bU, d, a, b)*binomial(l-1, lfInv)*dF*((1/2)*bF[j]*dU-(1/2)*dF*bU[j])*(-(1/2)*dF*dU+(1/2)*dot(aF, aU)+dot(bF, bU))
                        if coef != 0:
                            total += coef*invGW(dF, aF, bF)*OGW(dU, aU, bU, k)
    
    return (-2/b[j])*total

def OGW4(d,a,b,k):
    r = len(a)
    s = len(b)
    l = lU(d,a,b,k)
    if l != 0 or k<2:
        print("using OGW4 with ", d, a, b, k, "having l=", l)
        return 0
    total = 0

    for d1 in range(d+1):
        d2 = d+1 - d1
        if d1 <= d and d2 <= d:
            for a1 in SmallerMultiIndices(a):
                a2 = minus_list(a,a1)
                for b1 in SmallerMultiIndices(b):
                    b2 = minus_list(b,b1)
                    for k1 in range(k):
                        k2 = (k-1) - k1+1
                        l1 = lU(d1,a1,b1,k1)
                        l2 = lU(d2,a2,b2,k2)
                        if (d1,a1,b1,k1) != (d,a,b,k) and (d2,a2,b2,k2) != (d,a,b,k):
                            if IsInteger(l1) and IsInteger(l2):
                                coef = (1/2)*OOSign(d1, a1, b1, d2, a2, b2, d+1, a, b)*binomial(1, l1)*d1*((1/2)*binomial(k-2, k1-1)*d2-(1/2)*binomial(k-2, k1)*d1)
                                if coef != 0:
                                    total += coef*OGW(d1, a1, b1, k1)*OGW(d2, a2, b2, k2)
    for dU in range(d+1):
        dF = (d+1-dU)*(1/2)
        for aU in exeptionalDivisors(r) or aU in SmallerMultiIndices(a):
            aF = list(1/2 * np.array(minus_list(a,aU)))
            for bU in exeptionalDivisors(s) or bU in SmallerMultiIndices(a):
                bF = list(1/2 * np.array(minus_list(b,bU)))
                lu = lU(dU,aU,bU,k-1)
                lfInv = lFinv(dF,aF,bF)
                if IsInteger(dF) and AreIntegers(aF) and IsInteger(lu) and IsInteger(lfInv):
                    if dU >= 0 and dF >= 0 and IsNegative(aF) and IsNegative(bF) and lu >= 0 and lfInv >= 0:
                        coef = OCSign(dU, aU, bU, d+1, a, b)*binomial(1, lfInv)*dF**2*((1/2)*dF*dU-(1/2)*dot(aF, aU)-dot(bF, bU))
                        if coef != 0:
                            total -= coef*invGW(dF, aF, bF)*OGW(dU, aU, bU, k-1)
    
    if k == 2:
        total -= (1/4)*(d+1)**2*invGW((d+1)*(1/2), (1/2)*a, (1/2)*b)
    for d1 in range(1,d+1):
        d2 = (d+1)-d1
        for a1 in SmallerMultiIndices(a):
            a2 = minus_list(a,a1)
            for b1 in SmallerMultiIndices(b):
                b2 = minus_list(b,b1)
                for k1 in range(k):
                    k2 = k-k1
                    l1 = lU(d1,a1,b1,k1)
                    l2 = lU(d2,a2,b2,k2)
                    if (d1,a1,b1,k1) !=(d,a,b,k) and (d2,a2,b2,k2) != (d,a,b,k):
                        if IsInteger(l1) and IsInteger(l2):
                            coef = (1/2)*OOSign(d1, a1, b1, d2, a2, b2, d+1, a, b)*binomial(k-1, k1)*d1*((1/2)*binomial(0, l1-1)*d2-(1/2)*binomial(0, l1)*d1)
                            if coef != 0:
                                total -= coef*OGW(d1, a1, b1, k1)*OGW(d2, a2, b2, k2)
    for dU in range(1,d+1):
        dF = (d+1-dU)*(1/2)
        for aU in SmallerMultiIndices(a):
            aF = list(1/2 * np.array(minus_list(a,aU)))
            for bU in SmallerMultiIndices(b):
                bF = list(1/2 * np.array(minus_list(b,bU)))
                lu = lU(dU,aU,bU,k-1)
                lfInv = lFinv(dF,aF,bF)
                if IsInteger(dF) and AreIntegers(aF) and IsInteger(lu) and IsInteger(lfInv):
                    if dU >= 0 and dF >= 0 and IsNegative(aU) and IsNegative(aF) and IsNegative(bF) and lu >= 0 and lfInv >= 0:
                        coef = OCSign(dU, aU, bU, d+1, a, b)*dF*((1/2)*dF*dU-(1/2)*dot(aF, aU)-dot(bF, bU))*((1/2)*binomial(0, lfInv-1)*dU-binomial(0, lfInv)*dF)
                        if coef != 0:
                            total -= coef*invGW(dF, aF, bF)*OGW(dU, aU, bU, k-1)
    return total/(((d+1)*(1/4))*OOSign(1, Zeros(r), Zeros(s), d, a, b, d+1, a, b)*OGW(1, Zeros(r), Zeros(s), 0))
        

def OGW5a(d,a,b,k):
    r = len(a)
    s = len(b)
    l = lU(d,a,b,k)
    if l != 0 or not HasPositiveIndex(a) or d**2+(1-k)*d-k == 0:
        print("using OGW5a with ", d, a, b, k, "having l=", l, "division coefficient:", d**2+(1-k)*d-k)
        return 0
    
    i = 0
    while a[i] == 0:
        i += 1
    if i >= r:
        print("OGW5(", d, a, b, k, ") with l=", l)
        return 0
    total1 = 0
    for d1 in range(d+2):
        d2 = d+1-d1
        if d1 <= d and d2 <= d:
            for a1 in SmallerMultiIndices(a):
                a2 = minus_list(a,a1)
                for b1 in SmallerMultiIndices(b):
                    b2 = minus_list(b,b1)
                    for k1 in range(k+2):
                        k2 = (k+1) - k1 + 1
                        l1 = lU(d1,a1,b1,k1)
                        l2 = lU(d2,a2,b2,k2)
                        if (d1,a1,b1,k1) != (d,a,b,k) and (d2,a2,b2,k2) != (d,a,b,k):
                            if IsInteger(l1) and IsInteger(l2):
                                coef = (1/2)*OOSign(d1, a1, b1, d2, a2, b2, d+1, a, b)*binomial(0, l1)*d1*((1/2)*binomial(k, k1-1)*d2-(1/2)*binomial(k, k1)*d1)
                                if coef != 0:
                                    total1 = total1+coef*OGW(d1, a1, b1, k1)*OGW(d2, a2, b2, k2)
    for dU in range(d+1):
        dF = (d+1-dU)*(1/2)
        for aU in exeptionalDivisors(r) or aU in SmallerMultiIndices(a):
            aF = list(1/2 * np.array(minus_list(a,aU)))
            for bU in exeptionalDivisors(s) or bU in SmallerMultiIndices(b):
                bF = list(1/2 * np.array(minus_list(b,bU)))
                lu = lU(dU,aU,bU,k+1)
                lfInv = lFinv(dF,aF,bF)
                if IsInteger(dF) and AreIntegers(aF) and IsInteger(lu) and IsInteger(lfInv):
                    if (dU >=0) and (dF >= 0) and IsNegative(aF) and IsNegative(bF) and (lu >= 0) and (lfInv >= 0):
                        coef = OCSign(dU, aU, bU, d+1, a, b)*binomial(0, lfInv)*dF**2*((1/2)*dF*dU-(1/2)*dot(aF, aU)-dot(bF, bU))
                        if coef != 0:
                            total1 -= coef*invGW(dF, aF, bF)*OGW(dU, aU, bU, k+1)
    if k == 0:
        total1 -= (1/4)*(d+1)**2*invGW((d+1)*(1/2), (1/2)*a, (1/2)*b)


    total2 = 0
    for d1 in range(d+1):
        d2 = (d+1) - d1
        if d1 <= d and d2 <=d:
            for a1 in SmallerMultiIndices(a):
                a2 = minus_list(a,a1)
                for b1 in SmallerMultiIndices(b):
                    b2 = minus_list(b,b1)
                    for k1 in range(k+2):
                        k2 = (k+1) - k1 +1
                        l1 = lU(d1,a1,b1,k1)
                        l2 = lU(d2,a2,b2,k2)
                        if (d1,a1,b1,k1) != (d,a,b,k) and (d2,a2,b2,k2) != (d,a,b,k):
                            if IsInteger(l1) and IsInteger(l2):
                                coef = (1/2)*OOSign(d1, a1, b1, d2, a2, b2, d+1, a, b)*binomial(k+1, k1)*binomial(0, l1)*d1*(-(1/4)*d2*a1[i]+(1/4)*d1*a2[i])
                                if coef != 0:
                                    total2 += coef*OGW(d1, a1, b1, k1)*OGW(d2, a2, b2, k2)
    for dU in range(d+1):
        dF = (d+1-dU)*(1/2)
        for aU in exeptionalDivisors(r) or aU in SmallerMultiIndices(a):
            aF = list(1/2 * np.array(minus_list(a,aU)))
            for bU in exeptionalDivisors(s) or bU in SmallerMultiIndices(b):
                bF = list(1/2 * np.array(minus_list(b,bU)))
                lu = lU(dU,aU,bU,k+1)
                lfInv = lFinv(dF,aF,bF)
                if IsInteger(dF) and AreIntegers(aF) and IsInteger(lu) and IsInteger(lfInv):
                    if (dU >=0) and (dF >= 0) and IsNegative(aF) and IsNegative(bF) and (lu >= 0) and (lfInv >= 0):
                        coef = OCSign(dU, aU, bU, d+1, a, b)*binomial(0, lfInv)*dF*((1/2)*aF[i]*dU-(1/2)*dF*aU[i])*(-(1/2)*dF*dU+(1/2)*dot(aF, aU)+dot(bF, bU))
                        if coef  != 0:
                            total2 += coef*invGW(dF, aF, bF)*OGW(dU, aU, bU, k+1)

    total = total1+2*total2/a[i]
    return (4*(-1)**mu(d, a, b)/(d**2+(1-k)*d-k)*(1/2))*total

def OGW5b(d,a,b,k):
    r = len(a)
    s = len(b)
    l = lU(d,a,b,k)
    if l != 0 or not HasPositiveIndex(b) or d**2+(1-k)*d-k == 0:
        print("using OGW5b with ", d, a, b, k, "having l=", l, "division coefficient:", d**2+(1-k)*d-k)
        return 0
    
    j = 0
    while b[j] == 0:
        j += 1
    if j > s:
        print("OGW5b(", d, a, b, k, ") with l=", l)
        return 0
    total1 = 0

    for d1 in range(d+2):
        d2 = (d+1) -d1
        if d1 <= d and d2 <=d:
            if a1 in SmallerMultiIndices(a):
                a2 = minus_list(a,a1)
                for b1 in SmallerMultiIndices(b):
                    b2 = minus_list(b,b1)
                    for k1 in range(k+2):
                        k2 = (k+1) - k1+1
                        l1 = lU(d1,a1,b1,k1)
                        l2 = lU(d2,a2,b2,k2)
                        if (d1, a1, b1, k1) != (d, a, b, k) and (d2, a2, b2, k2) != (d, a, b, k):
                            if IsInteger(l1) and IsInteger(l2):
                                coef = (1/2)*OOSign(d1, a1, b1, d2, a2, b2, d+1, a, b)*binomial(0, l1)*d1*((1/2)*binomial(k, k1-1)*d2-(1/2)*binomial(k, k1)*d1)
                                if coef != 0:
                                    total1 += coef*OGW(d1, a1, b1, k1)*OGW(d2, a2, b2, k2)
    for dU in range(d+1):
        dF = (d+1-dU)*(1/2)
        for aU in exeptionalDivisors(r) or aU in SmallerMultiIndices(a):
            aF = list(1/2 * np.array(minus_list(a,aU)))
            for bU in exeptionalDivisors(s) or bU in SmallerMultiIndices(b):
                bF = list(1/2 * np.array(minus_list(b,bU)))
                lu = lU(dU,aU,bU,k+1)
                lfInv = lFinv(dF,aF,bF)
                if IsInteger(dF) and AreIntegers(aF) and IsInteger(lu) and IsInteger(lfInv):
                    if (dU >=0) and (dF >= 0) and IsNegative(aF) and IsNegative(bF) and (lu >= 0) and (lfInv >= 0):
                        coef = OCSign(dU, aU, bU, d+1, a, b)*binomial(0, lfInv)*dF**2*((1/2)*dF*dU-(1/2)*dot(aF, aU)-dot(bF, bU))
                        if coef != 0:
                            total1 -= coef*invGW(dF, aF, bF)*OGW(dU, aU, bU, k+1)
    if k == 0:
        total1 -=   (1/4)*(d+1)**2*invGW((d+1)*(1/2), (1/2)*a, (1/2)*b)
    total2 = 0
    for d1 in range(d+1):
        d2 = (d+1) -d1
        if d1 <= d and d2 <= d:
            for a1 in SmallerMultiIndices(a):
                a2 = minus_list(a,a1)
                for b1 in SmallerMultiIndices(b):
                    b2 = minus_list(b,b1)
                    for k1 in range(k+2):
                        k2 = (k+1) - k1+1
                        l1 = lU(d1,a1,b1,k1)
                        l2 = lU(d2,a2,b2,k2)
                        if (d1, a1, b1, k1) != (d, a, b, k) and (d2, a2, b2, k2) != (d, a, b, k):
                            if IsInteger(l1) and IsInteger(l2):
                                coef = (1/2)*OOSign(d1, a1, b1, d2, a2, b2, d+1, a, b)*binomial(k+1, k1)*binomial(0, l1)*d1*(-(1/4)*d2*b1[j]+(1/4)*d1*b2[j])
                                if coef != 0:
                                    total2 += coef*OGW(d1, a1, b1, k1)*OGW(d2, a2, b2, k2)
    
    for dU in range(d+1):
        dF = (d+1-dU)*(1/2)
        for aU in exeptionalDivisors(r) or aU in SmallerMultiIndices(a):
            aF = list(1/2 * np.array(minus_list(a,aU)))
            for bU in exeptionalDivisors(s) or bU in SmallerMultiIndices(b):
                bF = list(1/2 * np.array(minus_list(b,bU)))
                lu = lU(dU,aU,bU,k+1)
                lfInv = lFinv(dF,aF,bF)
                if IsInteger(dF) and AreIntegers(aF) and IsInteger(lu) and IsInteger(lfInv):
                    if (dU >=0) and (dF >= 0) and IsNegative(aF) and IsNegative(bF) and (lu >= 0) and (lfInv >= 0):
                        coef = OCSign(dU, aU, bU, d+1, a, b)*binomial(0, lfInv)*dF*((1/2)*bF[j]*dU-(1/2)*dF*bU[j])*(-(1/2)*dF*dU+(1/2)*dot(aF, aU)+dot(bF, bU))
                        if coef != 0:
                            total2 = total2+coef*invGW(dF, aF, bF)*OGW(dU, aU, bU, k+1)
    
    total = total1+2*total2/b[j]
    return (4*(-1)**mu(d, a, b)/(d**2+(1-k)*d-k)*(1/2))*total
@mem_dub
def OGW(d,alpha,beta,kappa):
    a = sorted(alpha)
    b = sorted(beta)
    k = kappa
    if lU(d,a,b,k) >= 0 and k >= 0:
        a = [x for x in a if x != 0]
        if a == []:
            a = [0]
        b = [x for x in b if x != 0]
        ones = len([x for x in b if x == 1])
        b = [x for x in b if x != 1]
        if b == []:
            b = [0]
        if ones > 0:
            return 2**ones*OGW(d, a, b, k)
    r = len(a)
    s = len(b)
    if not effectiveInv(d,a,b):
        return 0
    
    if d == 0:
        if k == 0 and a in exeptionalDivisors(r) and b == Zeros(s):
            return 2
        return 0
    
    if d == 1:
        if a == Zeros(r) and b == Zeros(s):
            if  k == 0:
                return 1
            if k == 2:
                return 2
            return 0
        if a in nexeptionalDivisors(r) and b == Zeros(s):
            if  k == 1:
                return 2
            return 0
        if a == Zeros(r) and b in nexeptionalDivisors(s):
            if k == 0:
                return 2
            return  0
    l = lU(d,a,b,k)
    if not IsInteger(l) or not AreIntegers(a) or not AreIntegers(b):
        return 0
    if l < 0 or k<0:
        return 0
    if a[0] == 1 and l >= 0 and k>= 0:
        return (-1)**(sum(a)+d)*OGW(d, minus_list(a,E(1, r)), b, k+1)
    if a[0] == 2 and l >= 0 and k>= 0:
        return -(1/2)*OGW(d, minus_list(a,list(2*np.array(E(1, r)))), b, k+2)+OGW(d, minus_list(a,list(2*np.array(E(1, r)))), b, k)
    if l >= 2:
        result1 = OGW1(d,a,b,k)
    if l >= 1 and k >= 1:
        result2 = OGW2(d,a,b,k)
    if l >= 1 and HasPositiveIndex(a):
        result3a = OGW3a(d,a,b,k)
    if l >= 1 and HasPositiveIndex(b):
        result3b = OGW3b(d,a,b,k)
    if l >=2 and k>=1:
        if result1 != result2:
            print("OGW1", d, a, b, k, "=", result1, " but OGW2=", result2)
    if l >= 2 and HasPositiveIndex(a):
        if result1 != result3a:
            print("OGW1", d, a, b, k, "=", result1, " but OGW3a=", result3a)
    if l >= 2 and HasPositiveIndex(b):
        if result1 != result3b:
            print("OGW1", d, a, b, k, "=", result1, " but OGW3b=", result3b)
    if l >= 1 and k >= 1 and HasPositiveIndex(a):
        if result2 != result3a:
            print("OGW2", d, a, b, k, "=", result2, " but OGW3a=", result3a)
    if l>=1 and k>=1 and HasPositiveIndex(b):
        if result2 != result3b:
            print("OGW2",d,a,b,k,"=",result2," but OGW3b=",result3b)
    if l >= 2:
        return result1
    if l >= 1 and k>=1:
        return result2
    if l >= 1 and HasPositiveIndex(a):
        return result3a
    if l >= 1 and HasPositiveIndex(b):
        return result3b
    
    if l == 0 and k >= 2:
        result1 = OGW4(d,a,b,k)
    if l == 0 and HasPositiveIndex(a) and d**2 + (1-k)*d - k != 0:
        result2 = OGW5a(d,a,b,k)
    if l == 0 and HasPositiveIndex(b) and d**2 + (1-k)*d - k != 0:
        result3 = OGW5b(d,a,b,k)
    if l == 0 and k >= 2 and HasPositiveIndex(a) and  d**2 + (1-k)*d - k != 0:
        if result1 != result2:
            print("OGW4", d, a, b, k, "=", result1, " but OGW5a=", result2)
                       
    if l == 0 and k >= 2 and HasPositiveIndex(b) and  d**2 + (1-k)*d - k != 0:
        if result2 != result3:
            print("OGW5a", d, a, b, k, "=", result2, " but OGW5b=", result3)
    if l == 0 and k>= 2:
        return result1
    if l == 0 and HasPositiveIndex(a) and d**2 + (1-k)*d - k != 0:
        return result2
    if l == 0 and HasPositiveIndex(b) and d**2 + (1-k)*d - k != 0:
        return result3
    
    if l <0:
        return 0

In [150]:
def OGW1(d,a,b,k):
    print(d)
    l = lU(d,a,b,k)
    if l < 2:
        print("OGW1(", d, a, k, ") with l=", l)
        return 0
    total = 0
    print(d)
    for d1 in range(1,d):
        d2 = d-d1
        for a1 in SmallerMultiIndices(a):
            a2 = minus_list(a,a1)
            for b1 in SmallerMultiIndices(b):
                b2 = minus_list(b,b1)
                for k1 in range(k+1):
                    k2 = k-k1+1
                    l1 = lU(d1,a1,b1,k1)
                    l2 = lU(d2,a2,b2,k2)
                    if IsInteger(l1) and IsInteger(l2):
                        coef = (1/2)*OOSign(d1, a1, b1, d2, a2, b2, d, a, b)*binomial(k, k1)*d1*((1/2)*binomial(l-2, l1-1)*d2-(1/2)*binomial(l-2, l1)*d1)
                        if coef != 0:
                            total += coef*OGW(d1, a1, b1, k1)*OGW(d2, a2, b2, k2)
    print(d)
    for dU  in range(1,d+1):
        dF = (d-dU)*(1/2)
        print(d,dU,dF)
        for aU in SmallerMultiIndices(a):
            aF = list(1/2*np.array(minus_list(a,aU)))
            for bU in SmallerMultiIndices(b):
                bF = list(1/2*np.array(minus_list(b,bU)))
                lu = lU(dU,aU,bU,k)
                lfInv = lFinv(dF,aF,bF)
                print(dF, aF, dU, aU, k, lu)
                if IsInteger(dF) and AreIntegers(aF) and IsInteger(lu) and IsInteger(lfInv):
                    if dU >= 0 and dF >= 0 and IsNegative(aU) and IsNegative(aF) and IsNegative(bF) and lu >= 0 and lfInv >= 0:
                        coef = OCSign(dU, aU, bU, d, a, b)*dF*((1/2)*dF*dU-(1/2)*dot(aF, aU)-dot(bF, bU))*((1/2)*binomial(l-2, lfInv-1)*dU-binomial(l-2, lfInv)*dF)
                        if coef != 0:
                            total += coef*invGW(dF, aF, bF)*OGW(dU, aU, bU, k)
                            print(dF, aF, dU, aU, k, lu)
    return total

In [151]:

OGW1(5, [2], [0], 8)




             



4 1 1.5
1.5 [0.0] 1 [0] 7 -3
4 2 1.0
1.0 [0.0] 2 [0] 7 -1
4 3 0.5
0.5 [0.0] 3 [0] 7 0
4 4 0.0
0.0 [0.0] 4 [0] 7 2
3 1 1.0
1.0 [0.0] 1 [0] 4 -1
3 2 0.5
0.5 [0.0] 2 [0] 4 0
3 3 0.0
0.0 [0.0] 3 [0] 4 2
2 1 0.5
0.5 [0.0] 1 [0] 0 1
2 2 0.0
0.0 [0.0] 2 [0] 0 2
2 1 0.5
0.5 [0.0] 1 [0] 1 0
2 2 0.0
0.0 [0.0] 2 [0] 1 2
3 1 1.0
1.0 [0.0] 1 [0] 3 -1
3 2 0.5
0.5 [0.0] 2 [0] 3 1
3 3 0.0
0.0 [0.0] 3 [0] 3 2
4 1 1.5
1.5 [0.0] 1 [0] 6 -2
4 2 1.0
1.0 [0.0] 2 [0] 6 -1
4 3 0.5
0.5 [0.0] 3 [0] 6 1
4 4 0.0
0.0 [0.0] 4 [0] 6 2
5 1 2.0
2.0 [1.0] 1 [0] 8 -3
2.0 [0.5] 1 [1] 8 -4
2.0 [0.0] 1 [2] 8 -4
5 2 1.5
1.5 [1.0] 2 [0] 8 -2
1.5 [0.5] 2 [1] 8 -2
1.5 [0.0] 2 [2] 8 -3
5 3 1.0
1.0 [1.0] 3 [0] 8 0
1.0 [0.5] 3 [1] 8 -1
1.0 [0.0] 3 [2] 8 -1
5 4 0.5
0.5 [1.0] 4 [0] 8 1
0.5 [0.5] 4 [1] 8 1
0.5 [0.0] 4 [2] 8 0
5 5 0.0
0.0 [1.0] 5 [0] 8 3
0.0 [0.5] 5 [1] 8 2
0.0 [0.0] 5 [2] 8 2


(-564+0j)